In [1]:
pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [2]:
pip install tensorflow==1.13.2

     |████████████████████████████████| 92.6MB 58kB/s 
     |████████████████████████████████| 3.2MB 43.9MB/s 
     |████████████████████████████████| 368kB 51.1MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0


In [3]:
pip install fasttext

     |████████████████████████████████| 71kB 1.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3014858 sha256=442c048c8bc11118ea0181316bb37ef846aa71947842f2168e9dd666c16c0868
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [1]:
import tensorflow as tf
import os
from matplotlib.image import imread
import numpy as np
import pandas as pd
from tensorflow.contrib import learn
import fasttext
import numpy as np
from sklearn.utils import shuffle
import re

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [0]:
class PreProcessing:
    def __init__(self,data_src):

        self.similar_pairs = self.build_corpus('Contrastive_loss_dataset.csv')
        self.embeddings_model = fasttext.train_unsupervised("text_corpus.txt", model='skipgram', lr=0.005, dim=64,
                                            ws=5, epoch=200)
        self.embeddings_model.save_model("ft_skipgram_ws5_dim64.bin")
        print('FastText training finished successfully.')
        self.current_index = 0
        input_X = list(self.similar_pairs['Possible Questions'])
        input_Y = list(self.similar_pairs['Atom'])
        wc_list_x = list(len(x.split(' ')) for x in input_X)
        wc_list_y = list(len(x.split(' ')) for x in input_Y)
        wc_list = []
        wc_list.extend(wc_list_x)
        wc_list.extend(wc_list_y)
        max_document_length = 16                 # or use a constant like 16, select this parameter based on your understanding of what could be a good choice
        number_of_elements = len(input_X)
        self.vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
        full_corpus = []
        full_corpus.extend(input_X)
        full_corpus.extend(input_Y)
        full_data = np.asarray(list(self.vocab_processor.fit_transform(full_corpus)))
        self.embeddings_lookup = []
        for word in list(self.vocab_processor.vocabulary_._mapping):
            try:
                self.embeddings_lookup.append(self.embeddings_model[str(word)])
            except:
                pass
        self.embeddings_lookup = np.asarray(self.embeddings_lookup)
        self.vocab_processor.save('./model_siamese_network/vocab')
        self.write_metadata(os.path.join('model_siamese_network','metadata.tsv'),list(self.vocab_processor.vocabulary_._mapping))
        print('Vocab processor executed and saved successfully.')
        self.X = full_data[0:number_of_elements]
        self.Y = full_data[number_of_elements:2*number_of_elements]
        self.label = list(self.similar_pairs['Relevant_question'])

    def preprocess(self,x):
        try:
            tk_x = x.lower()

            # list of characters which needs to be replaced with space
            space_replace_chars = ['?', ':', ',', '"', '[', ']', '~', '*', ';', '!', '?', '(', ')', '{', '}', '@', '$',
                                   '#', '.', '-', '/']
            tk_x = tk_x.translate({ord(x): ' ' for x in space_replace_chars})

            non_space_replace_chars = ["'"]
            tk_x = tk_x.translate({ord(x): '' for x in non_space_replace_chars})

            # remove non-ASCII chars
            tk_x = ''.join([c if ord(c) < 128 else '' for c in tk_x])

            # replace all consecutive spaces with one space
            tk_x = re.sub('\s+', ' ', tk_x).strip()

            # find all consecutive numbers present in the word, first converted numbers to * to prevent conflicts while replacing with numbers
            regex = re.compile(r'([\d])')
            tk_x = regex.sub('*', tk_x)
            nos = re.findall(r'([\*]+)', tk_x)
            # replace the numbers with the corresponding count like 123 by 3
            for no in nos:
                tk_x = tk_x.replace(no, "<NUMBER>", 1)

            return tk_x.strip().lower()
        except:
            return ""

    def build_corpus(self, filepath):
        similar_items = pd.read_csv(filepath)
        selected_cols = ['Possible Questions', 'Atom', 'Relevant_question']
        similar_items = similar_items[selected_cols]
        similar_items['Possible Questions'] = similar_items['Possible Questions'].apply(self.preprocess)
        similar_items['Atom'] = similar_items['Atom'].apply(self.preprocess)
        similar_items = shuffle(similar_items)
        similar_items = similar_items.drop_duplicates()
        question_list = list(similar_items['Possible Questions'])
        question_list.extend(list(similar_items['Atom']))
        pd.DataFrame(question_list).to_csv('text_corpus.txt', index=False)
        print('Text corpus generated and persisted successfully.')
        return similar_items

    def write_metadata(self,filename, labels):
        with open(filename, 'w') as f:
            f.write("Index\tLabel\n")
            for index, label in enumerate(labels):
                f.write("{}\t{}\n".format(index, label))

        print('Metadata file saved in {}'.format(filename))

    def get_siamese_batch(self, n):
        last_index = self.current_index
        self.current_index += n
        return self.X[last_index: self.current_index, :], self.Y[last_index: self.current_index, :], np.expand_dims(self.label[last_index: self.current_index], axis=1)


In [3]:
dataset = PreProcessing('Contrastive_loss_dataset.csv')

Text corpus generated and persisted successfully.
FastText training finished successfully.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Metadata file saved in model_siamese_network/metadata.tsv
Vocab processor executed and saved successfully.


In [0]:
class SiameseNetwork:

    def __init__(self, sequence_length, vocab_size, embedding_size, filter_sizes, num_filters, output_embedding_size, dropout_keep_prob,embeddings_lookup, l2_reg_lambda=0.0):
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.filter_sizes = filter_sizes
        self.num_filters = num_filters
        self.output_embedding_size = output_embedding_size
        self.dropout_keep_prob = dropout_keep_prob
        self.l2_loss = tf.constant(0.0)
        self.embeddings_lookup = embeddings_lookup

    def siamese_net(self, input_x, reuse=False):
        
        # Embedding layer
        with tf.variable_scope("embedding", reuse=reuse):
            self.W = tf.Variable(self.embeddings_lookup, name="W")
            self.word_embeddings = tf.nn.embedding_lookup(self.W, input_x)
            self.expanded_word_embeddings = tf.expand_dims(self.word_embeddings, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, self.embedding_size, 1, self.num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[self.num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.expanded_word_embeddings,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, self.sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = self.num_filters * len(self.filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total], name="pool_flat")

        # Add dropout [comment dropout during prediction]
        # with tf.name_scope("dropout"):
        #     h_drop = tf.nn.dropout(h_pool_flat, self.dropout_keep_prob, name="dropout")

        # Final output Layer
        with tf.variable_scope("output", reuse=reuse):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, self.output_embedding_size],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[self.output_embedding_size]), name="b2")
            out_net = tf.nn.xw_plus_b(self.h_pool_flat, W, b2, name="output_embedding")

        return out_net

    def contrastive_loss(self, model1, model2, y, margin):
        with tf.name_scope("contrastive-loss"):
            distance = tf.sqrt(tf.reduce_sum(tf.pow(model1 - model2, 2), 1, keepdims=True))
            return tf.reduce_mean(y * tf.square(distance) + (1 - y) * tf.square(tf.maximum((margin - distance), 0))) / 2 + 1e-9

In [0]:
# Model Hyper-parameters
embedding_dim = 64
filter_sizes = "3,4,5"
num_filters = 64
dropout_keep_prob = 0.5
l2_reg_lambda = 0.0
output_embedding_size = 64
batch_size = 32
train_iter = 500
step = 50
learning_rate = 0.0001
momentum = 0.99

In [0]:
model = SiameseNetwork(sequence_length=dataset.X.shape[1],
            vocab_size=len(dataset.vocab_processor.vocabulary_),
            embedding_size=embedding_dim,
            filter_sizes=list(map(int, filter_sizes.split(","))),
            num_filters=num_filters,
            output_embedding_size = output_embedding_size,
            dropout_keep_prob = dropout_keep_prob,
            embeddings_lookup= dataset.embeddings_lookup,
            l2_reg_lambda=l2_reg_lambda)

In [7]:
placeholder_shape = [None] + [dataset.X.shape[1]]
print("placeholder_shape", placeholder_shape)

placeholder_shape [None, 16]


In [0]:
# Setup Network
next_batch = dataset.get_siamese_batch
left_input = tf.placeholder(tf.int32, placeholder_shape, name='left_input')
right_input = tf.placeholder(tf.int32, placeholder_shape, name='right_input')

In [9]:
margin = 2.5
left_output = model.siamese_net(left_input, reuse=False)
right_output = model.siamese_net(right_input, reuse=True)
with tf.name_scope("similarity"):
    label = tf.placeholder(tf.int32, [None, 1], name='label')
    label_float = tf.to_float(label)
loss = model.contrastive_loss(left_output, right_output, label_float, margin)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [10]:
# Setup Optimizer
global_step = tf.Variable(0, trainable=False)

train_step = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True).minimize(loss,global_step=global_step)

Instructions for updating:
Use tf.cast instead.


In [0]:
from tensorflow.contrib.tensorboard.plugins import projector

In [12]:
# Start Training
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Setup Tensorboard
    tf.summary.scalar('step', global_step)
    tf.summary.scalar('loss', loss)
    for var in tf.trainable_variables():
        tf.summary.histogram(var.op.name, var)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('model_siamese_network', sess.graph)
    # adding embeddings to projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = "embedding/W"
    embed.metadata_path = "metadata.tsv"
    projector.visualize_embeddings(writer, config)
    print('Training...')
    # Batch Training
    for i in range(train_iter):
        batch_left, batch_right, batch_similarity_score = next_batch(batch_size)
        _, l, summary_str = sess.run([train_step, loss, merged],
                                     feed_dict={left_input: batch_left, right_input: batch_right, label: batch_similarity_score})
        writer.add_summary(summary_str, i)
        print("\r#%d - Loss" % i, l)
        
        if (i + 1) % step == 0 or l <= 1e-9:
            print('Saving model. Step: ',i)
            saver.save(sess, "model_siamese_network/model.ckpt")
        if l=='nan':
            break
    saver.save(sess, "model_siamese_network/model.ckpt")
print('Training completed successfully.')

Training...
#0 - Loss 1e-09
Saving model. Step:  0
#1 - Loss 1e-09
Saving model. Step:  1
#2 - Loss 1e-09
Saving model. Step:  2
#3 - Loss 1e-09
Saving model. Step:  3
#4 - Loss 1e-09
Saving model. Step:  4
#5 - Loss 1e-09
Saving model. Step:  5
#6 - Loss 1e-09
Saving model. Step:  6
#7 - Loss 1e-09
Saving model. Step:  7
#8 - Loss 1e-09
Saving model. Step:  8
#9 - Loss 1e-09
Saving model. Step:  9
#10 - Loss 1e-09
Saving model. Step:  10
#11 - Loss 1e-09
Saving model. Step:  11
#12 - Loss 1e-09
Saving model. Step:  12
#13 - Loss 1e-09
Saving model. Step:  13
#14 - Loss 1e-09
Saving model. Step:  14
#15 - Loss 1e-09
Saving model. Step:  15
#16 - Loss 1e-09
Saving model. Step:  16
#17 - Loss 1e-09
Saving model. Step:  17
#18 - Loss 1e-09
Saving model. Step:  18
#19 - Loss 1e-09
Saving model. Step:  19
#20 - Loss 1e-09
Saving model. Step:  20
#21 - Loss 1e-09
Saving model. Step:  21
#22 - Loss 1e-09
Saving model. Step:  22
#23 - Loss 1e-09
Saving model. Step:  23
#24 - Loss 1e-09
Saving 